## Evaluation Script

In [ ]:
import os 
import sys
from pathlib import Path
import yaml
from IPython.display import display, HTML

import pandas as pd
import numpy as np
import seaborn as sns
import torch
from speechbrain.utils.metric_stats import EER
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm

PATH_TO_HOME = '/home/' + os.getenv('USER') + '/adversarial-robustness-for-sr'
sys.path.append(PATH_TO_HOME)

from src.modules.metrics.metrics import VerificationMetrics
from src.modules.metrics.metrics import AS_norm 

os.chdir(f'{PATH_TO_HOME}')

## Experiment Params

In [ ]:
# Define tour experiment
# path_to_exp = f"{PATH_TO_HOME}/logs/train/runs/2025-04-22_18-05-30"
# path_to_exp = "/dataHDD/ahmad/hpc_results/libri_augmented3/vpc_amm_cyclic-available_models-max_dur10-bs32"
path_to_exp = "/dataHDD/ahmad/hpc_results/may3/vpc_amm_cyclic-available_models_LibriSpeech-max_dur10.0-bs32"  # /home/ahmad/adversarial-robustness-for-sr/logs/train/runs 

paths_to_metadata = f"{path_to_exp}/metadata"
paths_to_configs = f"{paths_to_metadata}/configs"
paths_to_src = f"{path_to_exp}/src"
os.chdir(f'{paths_to_configs}')

ckpt_path = [ckpt for ckpt in os.listdir(f"{path_to_exp}/checkpoints") if ckpt != 'last.ckpt']
assert len(ckpt_path) == 1, 'Expected a single model'
ckpt_path = f"{path_to_exp}/checkpoints/{ckpt_path[0]}"

In [ ]:
rel_path_eval = "../src/eval.py"
module = f"vpc.yaml"
datamodule = f"datasets/vpc.yaml"
# experiment = f"vpc/vpc_amm_cyclic_from_scratch.yaml"
experiment = f"vpc/vpc_amm_cyclic.yaml"

## Load Overrides and Modify

In [ ]:
overrides_path = Path("../../.hydra/overrides.yaml")

with open(overrides_path, 'r') as file:
    overrides_data_list = yaml.safe_load(file)

overrides_data = " ".join([f"'{item}'" if '{' in item and '}' in item else (item if 'logger' not in item else '') for item in overrides_data_list])
overrides_data

### Hot Fixes

In [ ]:
# 1. Update eval.py: test --> validate: metadata/src/eval.py

# 2. Commend defaults in experiment config: /metadata/configs/experiment/vpc/EXP.yaml

# 3. [for eval dev] Cohort Embeddings: train_data is None in metadata/src/datamodules/vpc_datamodule.py
# if stage == 'fit' or stage is None: --> if True:

# 4. [OPTIONAL - for eval] update: metadata/src/modules/multimodal_vpc.py
# override/write embedding type + set cohort_path = None


#=============
# clusters exps
overrides_data = overrides_data.replace("trainer.num_sanity_val_steps=0","")
overrides_data = overrides_data.replace("name=available_models_LibriSpeech/vpc_amm_cyclic-available_models_LibriSpeech-max_dur10.0-bs32", "")
overrides_data = overrides_data.replace("experiment=vpc/vpc_amm_cyclic", "")
overrides_data = overrides_data.replace("paths.data_dir=/tmp/1044973.tinygpu", "")
# overrides_data = overrides_data.replace("datamodule.loaders.train.batch_size=32", "")
overrides_data = overrides_data.replace("datamodule.loaders.valid.batch_size=32", "")
overrides_data = overrides_data.replace("paths.log_dir=/home/woody/dsnf/dsnf101h/results", f'paths.log_dir={path_to_exp}')
overrides_data = overrides_data.replace(" ckpt_path=/home/woody/dsnf/dsnf101h/results/train/runs/available_models_LibriSpeech/vpc_amm_cyclic-available_models_LibriSpeech-max_dur10.0-bs32/checkpoints/last.ckpt", "")
overrides_data

In [ ]:
env_file_path = f"{paths_to_metadata}/.env"

if not os.path.exists(env_file_path):
    with open(env_file_path, 'w') as env_file:
        env_file.write(f"PROJECT_ROOT={paths_to_metadata}/adversarial-robustness-for-sr\n")
        env_file.write("NEPTUNE_API_TOKEN=eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1N2E3OGMzMC1mYWE5LTQzMzctYjBjZC1mN2I0ZmRhN2FjMjcifQ==\n")
        env_file.write("WANDB_API_TOKEN=21213080edcfbf4ca6154c143ec533a3b2eb74c0\n")

In [ ]:
# !python $rel_path_eval module=$module datamodule=$datamodule experiment=$experiment paths.log_dir=logs ckpt_path=$ckpt_path 
# !python $rel_path_eval experiment=$experiment test=true train=false paths.log_dir=logs ckpt_path=$ckpt_path

# !python $rel_path_eval module=$module experiment=$experiment datamodule=$datamodule $overrides_data trainer=gpu ckpt_path=$ckpt_path paths.data_dir=$PATH_TO_HOME/data \
# module.scores_norm.embeds_metric_params.cohort_per_model=40000 module.scores_norm.scores_norm_params.min_cohort_size=10

!python $rel_path_eval +experiment=$experiment $overrides_data ckpt_path=$ckpt_path paths.data_dir=$PATH_TO_HOME/data \
module.scores_norm.embeds_metric_params.cohort_per_model=40000 module.scores_norm.scores_norm_params.topk=1000 module.scores_norm.scores_norm_params.min_cohort_size=10